In [8]:
import pandas as pd
import time
import random
import tkinter as tk
from tkinter import filedialog
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Function to initialize the Chrome driver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def get_databridge_ranking(driver, keyword):
    search_url = "https://www.google.com/search?q=" + keyword
    driver.get(search_url)
    
    time.sleep(random.uniform(2, 4))  # Wait between 2 to 4 seconds
    
    try:
        results = driver.find_elements(By.CSS_SELECTOR, 'div.g')
        rank = 1
        for result in results:
            link = result.find_element(By.TAG_NAME, 'a').get_attribute('href')
            if 'databridgemarketresearch.com' in link:
                return rank, link
            rank += 1
    except Exception as e:
        print(f"An error occurred: {e}")
    return None, None

# Use Tkinter to open a file dialog
root = tk.Tk()
root.withdraw()  # Hide the root window
file_path = filedialog.askopenfilename(title="Select Excel File", filetypes=[("Excel files", "*.xlsx")])

if file_path:
    driver = initialize_driver()
    
    try:
        # Load the keywords from the selected Excel file
        keywords_df = pd.read_excel(file_path)

        # Print column names for debugging
        print("Columns in the Excel file:", keywords_df.columns)
        
        # Ensure the column name matches exactly with the one in the Excel file
        keyword_column_name = 'Keywords'  # Adjusted to match the column name in your file
        
        if keyword_column_name not in keywords_df.columns:
            raise ValueError(f"Column '{keyword_column_name}' not found in the Excel file.")
        
        # Add new columns for the ranking and URL
        keywords_df['Ranking'] = None
        keywords_df['URL'] = None

        # Loop through each keyword and get the ranking and URL
        for i, keyword in enumerate(keywords_df[keyword_column_name]):
            print(f"Processing keyword {i+1}/{len(keywords_df)}: {keyword}")
            rank, url = get_databridge_ranking(driver, keyword)
            
            # Store the ranking and URL in the DataFrame
            keywords_df.at[i, 'Ranking'] = rank
            keywords_df.at[i, 'URL'] = url
            
            # Print the results to the debug screen
            print(f"Keyword: {keyword}, Ranking: {rank}, URL: {url}")
            
            # Wait a random time between 5 to 10 seconds before the next search
            time.sleep(random.uniform(5, 10))
        
        # Save the final results back to Excel
        save_path = file_path.replace('.xlsx', '_with_ranking.xlsx')
        keywords_df.to_excel(save_path, index=False)
        print(f"Results saved to {save_path}")
    
    except Exception as e:
        print(f"An error occurred during processing: {e}")
        
        # Save the current progress to a new file if an error occurs
        error_file_path = file_path.replace('.xlsx', '_error_storage.xlsx')
        keywords_df.to_excel(error_file_path, index=False)
        print(f"Progress saved to {error_file_path}")
    
    finally:
        # Close the driver
        driver.quit()
else:
    print("No file selected.")


Columns in the Excel file: Index(['Keywords'], dtype='object')
Processing keyword 1/18:  Building Management System Market
Keyword:  Building Management System Market, Ranking: 2, URL: https://www.databridgemarketresearch.com/reports/global-building-management-system-market
Processing keyword 2/18:  Abrasives Market
Keyword:  Abrasives Market, Ranking: 14, URL: https://www.databridgemarketresearch.com/reports/global-abrasive-market
Processing keyword 3/18:  Active Pharmaceutical Ingredients (API) Market
Keyword:  Active Pharmaceutical Ingredients (API) Market, Ranking: 8, URL: https://www.databridgemarketresearch.com/reports/global-active-pharmaceutical-ingredient-api-market
Processing keyword 4/18:  Advanced Wound Care Market
Keyword:  Advanced Wound Care Market, Ranking: 10, URL: https://www.databridgemarketresearch.com/reports/global-advanced-wound-care-market
Processing keyword 5/18:  Aesthetic & Cosmetic Surgery Devices Market   
Keyword:  Aesthetic & Cosmetic Surgery Devices Mark